In [1]:
%pip install --upgrade pip 
%pip install --upgrade transformers datasets[audio] accelerate

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.

In [2]:
import os
os.environ["PATH"] += os.pathsep + r"C:\github\gpt_agent_2025_easyspub\ffmpeg-2025-02-10-full_build\bin"

In [4]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,   # 청크별로 타임스탬프 반환
    chunk_length_s=10,  # 입력 오디오 10초씩 나누기r
    stride_length_s=2,  # 2초씩 겹치도록 청크 나누기
) 

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = "../audio/lsy_audio_2023_58s.mp3"

result = pipe(sample)
# print(result["text"])

print(result)


Device set to use cuda:0
c:\github\gpt_agent_2025_easyspub\venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexp

{'text': ' 안녕하세요. 이 강의는 GPT API로 챗봇 만들기 라는 내용을 다루는 강의입니다. GPT API에 대해서 생소하신 분들도 있을텐데 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요. 그래서 이런 강의들이 사실 많이 있습니다. 그래서 여러 가지들이 있는데 이 강의 특징이라고 한다면 GPT로 명확한 미션을 달성하는 챕터 프로그램을 만드는게 사실 쉽지는 않은데 이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할 거고요. 그 예제로 예제는 여러가지가 될 수 있는데 여기서 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 것을 다루려고 합니다. 그래서 프로그램이 실행되는 모습을 한번 보여드릴게요. 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고', 'chunks': [{'timestamp': (0.0, 6.3), 'text': ' 안녕하세요. 이 강의는 GPT API로 챗봇 만들기 라는 내용을 다루는 강의입니다.'}, {'timestamp': (7.18, 10.0), 'text': ' GPT API에 대해서 생소하신 분들도 있을텐데'}, {'timestamp': (11.0, 17.0), 'text': ' 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서'}, {'timestamp': (17.0, 21.0), 'text': ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요.'}, {'timestamp': (21.0, 24.0), 'text': ' 그래서 이런 강의들이 사실 많이 있습니다.'}, {'timestamp': (24.0, 27.48), 'text': ' 그래서 여러 가지들이 있는데 이 강의 특징이라고 한다면'}, {'timestamp': (27.48, 29.58), 'text': ' GPT로 명확한 미션을 달성하는'}, {'timestamp

In [5]:
# chunks를 CSV 파일로 저장
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_audio_2023_58.csv", index=False, sep="|")
display(df)


,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT API로 챗봇 만들기 라는 내용을 다루는 강의입니다.
1,7.18,10.00,GPT API에 대해서 생소하신 분들도 있을텐데
2,11.00,17.00,"우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서"
3,17.00,21.00,우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요.
4,21.00,24.00,그래서 이런 강의들이 사실 많이 있습니다.
5,24.00,27.48,그래서 여러 가지들이 있는데 이 강의 특징이라고 한다면
6,27.48,29.58,GPT로 명확한 미션을 달성하는
7,29.58,31.66,챕터 프로그램을 만드는게 사실
8,31.66,34.32,쉽지는 않은데 이걸 어떻게 해서
9,34.32,36.40,구현을 하는지 그리고 그게 왜 필요한지에 대해서
